In [6]:
# Colab 환경에서 pip를 업그레이드하고, 호환 버전으로 torch와 transformers 설치
!pip install --upgrade pip

# PyTorch 2.0 (CUDA 11.8) + Transformers 최신버전 예시
!pip install --upgrade \
  torch==2.0.0+cu118 torchvision==0.15.0+cu118 torchaudio==2.0.0+cu118 \
  transformers \
  --extra-index-url https://download.pytorch.org/whl/cu118

import sys
sys.exit("런타임을 다시 시작합니다. 런타임 재시작 후 노트북을 재실행하세요.")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 18.3 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 GB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 32.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 86.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.3/63.3 MB 109.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 99.2 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.48.3
    Uninstalling transformers-4.48.3:
      Successfully uninstalled transformers-4.48.3
  Attempting uninstall: triton
    Found existing installation: triton 3.1.0
    Uninstalling triton-3.1.0:
      Succe

SystemExit: 런타임을 다시 시작합니다. 런타임 재시작 후 노트북을 재실행하세요.

/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [1]:
from huggingface_hub import login

login()

In [5]:
import torch
import numpy as np
from transformers import AutoModel, AutoTokenizer

def save_digit_embeddings(model_name="google/gemma-2b", save_path="gemma_embeddings.npz"):
    # 1. 모델과 토크나이저 불러오기
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModel.from_pretrained(model_name)
    model.eval()  # 추론 모드

    # 2. 모델의 입력 임베딩 레이어 가져오기
    embedding_layer = model.get_input_embeddings()  # nn.Embedding 모듈

    # 3. "0" ~ "9" 각 문자에 대한 임베딩 추출
    embeddings_dict = {}
    for digit in map(str, range(10)):
        # 토큰화 (특수 토큰 없이)
        inputs = tokenizer(digit, add_special_tokens=False, return_tensors="pt")
        input_ids = inputs["input_ids"]  # shape: (batch_size=1, seq_len)

        with torch.no_grad():
            # 임베딩 레이어에서 해당 토큰의 벡터를 직접 추출
            embedded = embedding_layer(input_ids)  # shape: (1, seq_len, hidden_size)
            # 보통 "0"과 같은 단일 문자라면 seq_len=1이므로 아래처럼 squeeze해도 됨
            embedding_vector = embedded.squeeze(0).squeeze(0).cpu().numpy()

        # 딕셔너리에 저장
        embeddings_dict[digit] = embedding_vector

    # 4. .npz 파일로 저장 (딕셔너리를 한 번에 저장)
    np.savez(save_path, data=embeddings_dict)
    print(f"Saved digit embeddings to {save_path}")


if __name__ == "__main__":
    save_digit_embeddings()


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Saved digit embeddings to gemma_embeddings.npz


In [6]:
import numpy as np

def load_digit_embeddings(file_path="digit_embeddings.npz"):
    # allow_pickle=True 필요
    data = np.load(file_path, allow_pickle=True)
    embeddings_dict = data["data"].item()  # 저장 시 "data"라는 키로 딕셔너리 보관
    return embeddings_dict

embeddings = load_digit_embeddings("gemma_embeddings.npz")
print("임베딩 벡터 (문자 '0'):", embeddings["0"])
print("임베딩 벡터 (문자 '9'):", embeddings["9"])


임베딩 벡터 (문자 '0'): [ 0.1328125  -0.09863281 -0.12402344 ... -0.01452637  0.001091
  0.02050781]
임베딩 벡터 (문자 '9'): [ 0.11230469 -0.10302734 -0.10449219 ... -0.0189209  -0.01531982
  0.00793457]
